In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from mxnet import nd, autograd, gluon
import mxnet as mx
from mxnet.gluon import nn, rnn

In [2]:
def n(digits =3):
    number = ''
    for i in range(np.random.randint(1, digits + 1)):
        number += np.random.choice(list('0123456789'))
    return int(number)

def padding(chars, maxlen):
    return chars + ' ' * (maxlen - len(chars))

N = 50000
N_train = int(N * .9)
N_validation = N - N_train

digits = 3
input_digits = digits * 2 + 3
output_digits = digits + 3

added = set()
questions = []
answers = []
answers_y = []

while len(questions) < N:
    a, b = n(), n()
    pair = tuple(sorted((a, b)))
    if pair in added:
        continue
        
    question = 'S{}+{}E'.format(a, b)
    question = padding(question, input_digits)
    answer = 'S' + str(a + b) + 'E'
    answer = padding(answer, output_digits)
    answer_y = str(a + b) + 'E'
    answer_y = padding(answer_y, output_digits)
    
    added.add(pair)
    questions.append(question)
    answers.append(answer)
    answers_y.append(answer_y)
    
chars = '0123456789+SE '
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

X = np.zeros((len(questions), input_digits, len(chars)), dtype=np.integer)
Y = np.zeros((len(questions), digits + 3, len(chars)), dtype=np.integer)
Z = np.zeros((len(questions), digits + 3, len(chars)), dtype=np.integer)

for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers[i]):
        Y[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers_y[i]):
        Z[i, t, char_indices[char]] = 1
    
X_train, X_validation, Y_train, Y_validation, Z_train, Z_validation = \
    train_test_split(X, Y, Z, train_size=N_train)

/opt/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Testset

In [3]:
def gen_n_test(N):
    q = []
    y = []
    for i in range(N):
        a, b = n(), n() 
        question = '{}+{}'.format(a, b)
        answer_y = str(a + b)
        q.append(question)
        y.append(answer_y)
    return(q,y)

In [4]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [5]:
class calculator(gluon.Block):
    def __init__(self, n_hidden, in_seq_len, out_seq_len, vocab_size, **kwargs):
        super(calculator, self).__init__(**kwargs)
        self.in_seq_len = in_seq_len
        self.out_seq_len = out_seq_len
        self.n_hidden = n_hidden
        self.vocab_size = vocab_size
        
        with self.name_scope():
            self.encoder = rnn.LSTMCell(hidden_size = n_hidden)
            self.decoder = rnn.LSTMCell(hidden_size = n_hidden)
            self.batchnorm = nn.BatchNorm(axis = 2)
            self.dense = nn.Dense(self.vocab_size, flatten = False)
            
    def forward(self, inputs, outputs):
        # Since we don't use intermediate states for 'though vector', we don't need to unroll it.
        # In the later examples, we will use LSTM class rather than LSTMCell class.
        enout, (next_h, next_c) = self.encoder.unroll(inputs = inputs
                                                    , length = self.in_seq_len
                                                    , merge_outputs = True)
        for i in range(self.out_seq_len):
            deout, (next_h, next_c) = self.decoder(outputs[:, i, :], [next_h, next_c],)
            if i == 0:
                deouts = deout
            else:
                deouts = nd.concat(deouts, deout, dim = 1)
            #print('i= {}, deouts= {}'.format(i, deouts.shape))
        
        deouts = nd.reshape(deouts, (-1, self.out_seq_len, self.n_hidden))
        deouts = self.batchnorm(deouts)
        deouts_fc = self.dense(deouts)
        return deouts_fc
    
    def calculation(self, input_str, char_indices, indices_char, input_digits = 9, lchars = 14, ctx = mx.gpu()):
        input_str = 'S' + input_str + 'E'
        X = nd.zeros((1, input_digits, lchars), ctx = ctx)
        for t, char in enumerate(input_str):
            X[0, t, char_indices[char]] = 1
        Y_init = nd.zeros((1, lchars), ctx = ctx)
        Y_init[0, char_indices['S']] = 1
        enout, (next_h, next_c) = self.encoder.unroll(inputs = X, length = self.in_seq_len, merge_outputs = True)
        deout = Y_init
        
        for i in range(self.out_seq_len):
            deout, (next_h, next_c) = self.decoder(deout, [next_h, next_c])
            #print('dim deout = {}'.format(deout.shape))
            deout = nd.expand_dims(deout, axis = 1)
            #print('dim deout = {}'.format(deout.shape))
            deout = self.batchnorm(deout)
            deout = deout[:, 0, :]
            #print('dim deout = {}'.format(deout.shape))

            deout_sm = self.dense(deout)
            deout = nd.one_hot(nd.argmax(nd.softmax(deout_sm, axis = 1), axis = 1), depth = self.vocab_size)
            if i == 0:
                ret_seq = indices_char[nd.argmax(deout_sm, axis = 1).asnumpy()[0].astype('int')]
            else:
                ret_seq += indices_char[nd.argmax(deout_sm, axis = 1).asnumpy()[0].astype('int')]

            if ret_seq[-1] == ' ' or ret_seq[-1] == 'E':
                break
        return ret_seq.strip('E').strip()
        

In [6]:
tr_set = gluon.data.ArrayDataset(X_train, Y_train, Z_train)
tr_data_iterator = gluon.data.DataLoader(tr_set, batch_size=256, shuffle=True)

te_set =gluon.data.ArrayDataset(X_validation, Y_validation, Z_validation)
te_data_iterator = gluon.data.DataLoader(te_set, batch_size=256, shuffle=True)

ctx = mx.gpu()
model = calculator(300, 9, 6, 14)
model.collect_params().initialize(mx.init.Xavier(), ctx = ctx)

trainer = gluon.Trainer(model.collect_params(), 'rmsprop')
loss = gluon.loss.SoftmaxCrossEntropyLoss(axis = 2, sparse_label = False)

In [7]:
print(model)

calculator(
  (encoder): LSTMCell(None -> 1200)
  (decoder): LSTMCell(None -> 1200)
  (batchnorm): BatchNorm(axis=2, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=None)
  (dense): Dense(None -> 14, linear)
)


In [8]:
def calculate_loss(model, data_iter, loss_obj, ctx = ctx):
    test_loss = []
    for i, (x_data, y_data, z_data) in enumerate(data_iter):
        x_data = x_data.as_in_context(ctx).astype('float32')
        y_data = y_data.as_in_context(ctx).astype('float32')
        z_data = z_data.as_in_context(ctx).astype('float32')
        with autograd.predict_mode():
            z_output = model(x_data, y_data)
            loss_te = loss_obj(z_output, z_data)
        curr_loss = nd.mean(loss_te).asscalar()
        test_loss.append(curr_loss)
    return np.mean(test_loss)

In [9]:
epochs = 201

tot_test_loss = []
tot_train_loss = []
for e in range(epochs):
    train_loss = []
    for i, (x_data, y_data, z_data) in enumerate(tr_data_iterator):
        x_data = x_data.as_in_context(ctx).astype('float32')
        y_data = y_data.as_in_context(ctx).astype('float32')
        z_data = z_data.as_in_context(ctx).astype('float32')
        
        with autograd.record():
            z_output = model(x_data, y_data)
            loss_ = loss(z_output, z_data)
        loss_.backward()
        trainer.step(x_data.shape[0])
        curr_loss = nd.mean(loss_).asscalar()
        train_loss.append(curr_loss)
        
    if e % 10 == 0:
        q, y = gen_n_test(10)
        for i in range(10):
            with autograd.predict_mode():
                p = model.calculation(q[i], char_indices, indices_char).strip()
                iscorr = 1 if p == y[i] else 0
                if iscorr == 1:
                    print(colors.ok + '☑' + colors.close, end=' ')
                else:
                    print(colors.fail + '☒' + colors.close, end=' ')
                print("{} = {}({}) 1/0 {}".format(q[i], p, y[i], str(iscorr) ))
    #caculate test loss
    test_loss = calculate_loss(model, te_data_iterator, loss_obj = loss, ctx=ctx) 

    print("Epoch %s. Train Loss: %s, Test Loss : %s" % (e, np.mean(train_loss), test_loss))    
    tot_test_loss.append(test_loss)
    tot_train_loss.append(np.mean(train_loss))


☒ 93+0 = 1000(93) 1/0 0
☒ 0+43 = 449(43) 1/0 0
☒ 864+752 = 1009(1616) 1/0 0
☒ 175+82 = 1009(257) 1/0 0
☒ 761+8 = 1039(769) 1/0 0
☒ 90+421 = 1009(511) 1/0 0
☒ 446+788 = 1039(1234) 1/0 0
☒ 62+36 = 630(98) 1/0 0
☒ 4+4 = 444(8) 1/0 0
☒ 617+6 = 779(623) 1/0 0
Epoch 0. Train Loss: 1.1954819, Test Loss : 1.1388888
Epoch 1. Train Loss: 1.1225086, Test Loss : 1.1047744
Epoch 2. Train Loss: 1.0937438, Test Loss : 1.0658474
Epoch 3. Train Loss: 1.0264179, Test Loss : 0.97517955
Epoch 4. Train Loss: 0.9184651, Test Loss : 0.86142606
Epoch 5. Train Loss: 0.82754004, Test Loss : 0.7985453
Epoch 6. Train Loss: 0.7551264, Test Loss : 0.77629375
Epoch 7. Train Loss: 0.6870809, Test Loss : 0.66588545
Epoch 8. Train Loss: 0.6154428, Test Loss : 0.6293352
Epoch 9. Train Loss: 0.5184007, Test Loss : 0.4856955
☒ 53+905 = 968(958) 1/0 0
☒ 3+7 = 141(10) 1/0 0
☑ 50+7 = 57(57) 1/0 1
☒ 3+66 = 70(69) 1/0 0
☑ 73+166 = 239(239) 1/0 1
☒ 111+472 = 563(583) 1/0 0
☑ 0+821 = 821(821) 1/0 1
☒ 829+471 = 1280(1300) 1/0 0
☒

☑ 61+976 = 1037(1037) 1/0 1
☑ 0+97 = 97(97) 1/0 1
☑ 1+1 = 2(2) 1/0 1
☑ 9+25 = 34(34) 1/0 1
☒ 79+0 = 89(79) 1/0 0
☒ 46+6 = 62(52) 1/0 0
☑ 231+150 = 381(381) 1/0 1
☑ 2+7 = 9(9) 1/0 1
☑ 50+79 = 129(129) 1/0 1
☑ 768+99 = 867(867) 1/0 1
Epoch 90. Train Loss: 0.0027768165, Test Loss : 0.016044449
Epoch 91. Train Loss: 0.0018821121, Test Loss : 0.006778906
Epoch 92. Train Loss: 0.0036900977, Test Loss : 0.015098724
Epoch 93. Train Loss: 0.002596438, Test Loss : 0.006146586
Epoch 94. Train Loss: 0.0019872563, Test Loss : 0.0064983903
Epoch 95. Train Loss: 0.0017905737, Test Loss : 0.008139258
Epoch 96. Train Loss: 0.0016750622, Test Loss : 0.0075414246
Epoch 97. Train Loss: 0.002756147, Test Loss : 0.00995801
Epoch 98. Train Loss: 0.0017337408, Test Loss : 0.006174893
Epoch 99. Train Loss: 0.0022295276, Test Loss : 0.011258623
☑ 8+8 = 16(16) 1/0 1
☑ 38+648 = 686(686) 1/0 1
☑ 3+55 = 58(58) 1/0 1
☑ 98+598 = 696(696) 1/0 1
☑ 29+25 = 54(54) 1/0 1
☑ 75+91 = 166(166) 1/0 1
☑ 32+9 = 41(41) 1/0 1
☑ 1+

Epoch 176. Train Loss: 2.7230071e-06, Test Loss : 0.0032036384
Epoch 177. Train Loss: 2.6737152e-06, Test Loss : 0.0033535666
Epoch 178. Train Loss: 2.6182997e-06, Test Loss : 0.0032702852
Epoch 179. Train Loss: 2.6338332e-06, Test Loss : 0.003262053
☑ 49+42 = 91(91) 1/0 1
☑ 357+99 = 456(456) 1/0 1
☒ 7+80 = 88(87) 1/0 0
☑ 415+79 = 494(494) 1/0 1
☑ 770+4 = 774(774) 1/0 1
☑ 52+59 = 111(111) 1/0 1
☑ 233+4 = 237(237) 1/0 1
☑ 0+6 = 6(6) 1/0 1
☑ 34+33 = 67(67) 1/0 1
☑ 54+88 = 142(142) 1/0 1
Epoch 180. Train Loss: 2.621395e-06, Test Loss : 0.0031281826
Epoch 181. Train Loss: 2.5484942e-06, Test Loss : 0.0034650187
Epoch 182. Train Loss: 2.5050192e-06, Test Loss : 0.003182826
Epoch 183. Train Loss: 2.4409676e-06, Test Loss : 0.0033390813
Epoch 184. Train Loss: 2.4870913e-06, Test Loss : 0.0032807663
Epoch 185. Train Loss: 2.4206333e-06, Test Loss : 0.0032118172
Epoch 186. Train Loss: 2.4492713e-06, Test Loss : 0.003574138
Epoch 187. Train Loss: 2.4270032e-06, Test Loss : 0.003264118
Epoch 188.